In [8]:
import geopandas as gpd

In [11]:
cities = ["福州", "厦门", "漳州", "泉州", "宁德", "莆田"]  # 所有城市列表
width = '200'
area_dic = {}
for area in cities:
    Area_path = rf"H:\研究区\格网文件\200\{area}.shp"  # 城市格网矢量文件路径
    GDF = gpd.read_file(Area_path)
    area_dic[area] = max(GDF["FID"]) + 1

In [12]:
area_dic_ = {}
area_dic_[width] = area_dic
area_dic_

{'200': {'福州': 366050,
  '厦门': 48959,
  '漳州': 380951,
  '泉州': 344496,
  '宁德': 413610,
  '莆田': 121124}}

In [ ]:
GDF = gpd.read_file("H:\研究区\村级\福州.shp")
max(GDF["FID"].values)

In [1]:
import os
import glob
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from easydict import EasyDict as edict
import numpy as np
from itertools import product

od_df = pd.read_csv(r"H:\结果数据\OD流量\村级\厦门\4月\Timing_4_1.csv")
do_df = pd.read_csv(r"H:\结果数据\DO流量\村级\厦门\4月\Timing_4_1.csv")

o_flow_df = od_df[['O_id', 'time', 'flow']].groupby(['O_id', 'time'])['flow'].sum().reset_index(name='O_flow')
d_flow_df = do_df[['D_id', 'time', 'flow']].groupby(['D_id', 'time'])['flow'].sum().reset_index(name='D_flow')

# 2. 重命名以统一字段名
o_flow_df = o_flow_df.rename(columns={'O_id': 'id'})
d_flow_df = d_flow_df.rename(columns={'D_id': 'id'})
all_ids = [i for i in range(712)]
all_times = [i for i in range(96)]
# 3. 构建完整时间 × id 组合
full_index = pd.DataFrame(product(all_ids, all_times), columns=['id', 'time'])

# 4. 合并 O_flow 和 D_flow，并填 0
o_full = full_index.merge(o_flow_df, on=['id', 'time'], how='left').fillna({'O_flow': 0})
d_full = full_index.merge(d_flow_df, on=['id', 'time'], how='left').fillna({'D_flow': 0})

# 5. 合并 O_flow 和 D_flow
merged_df = pd.merge(o_full[['id', 'time', 'O_flow']],
                    d_full[['id', 'time', 'D_flow']],
                    on=['id', 'time'],
                    how='outer')

# 6. 填充缺失值（极端情况下）
merged_df['O_flow'] = merged_df['O_flow'].fillna(0)
merged_df['D_flow'] = merged_df['D_flow'].fillna(0)

# 7. 排序并重置索引
merged_df = merged_df.sort_values(by=['id', 'time']).reset_index(drop=True)

In [7]:
od_df[od_df['time']==3 ]

,O_id,D_id,time,flow
295,0,663,3,1
438,2,2,3,2
878,2,278,3,1
2286,2,676,3,1
2320,2,679,3,1
...,...,...,...,...
598459,704,703,3,2
598543,704,704,3,6
599240,705,491,3,1
599755,705,605,3,1


In [4]:
d_full[]

,id,time,D_flow
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
68347,711,91,1.0
68348,711,92,0.0
68349,711,93,0.0
68350,711,94,0.0


In [6]:
merged_df.head(50)

,id,time,O_flow,D_flow
0,0,0,0.0,0.0
1,0,1,0.0,0.0
2,0,2,0.0,0.0
3,0,3,1.0,0.0
4,0,4,1.0,0.0
5,0,5,3.0,0.0
6,0,6,1.0,0.0
7,0,7,2.0,0.0
8,0,8,0.0,0.0
9,0,9,1.0,2.0
